# Parte 3 - Resolvendo Cadeia de Markov de Tempo Contínuo

In [3]:
import numpy as np
import pandas as pd
import networkx as nx
import random as rd

In [ ]:
tx_falha = 1
tx_conserto = 1

In [79]:
grafo_base = np.zeros((5, 5))
arestas = [(1,0),(2,1),(3,2),(4,3),(4,0),(4,2)]

for aresta in arestas:
    grafo_base[aresta] = 1

grafo_base

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 1., 1., 0.]])

## Cenário 1 - Reparo Paralelo

Listando Estados Possíveis

In [80]:
import itertools

possibilities = list()
arestas = nx.Graph(grafo_base).edges

for i in range(len(arestas)+1):
    for conjunto in itertools.combinations(arestas, i):
        possibilities.append( list(conjunto) )

print(len(possibilities))

64


Gerando Matriz Q

In [58]:
Q = np.zeros((len(possibilities),len(possibilities)))

total_arestas = len(arestas)

for i in range(0, len(possibilities)):
    estado_i = set(possibilities[i])
    for j in range(0, len(possibilities)):
        estado_j = set(possibilities[j])

        if i == j:
            continue

        if len(estado_j - estado_i) == 1:
            # Houve uma falha
            Q[i,j] = tx_falha
        elif len(estado_i - estado_j) == 1:
            # Houve um conserto
            Q[i,j] = tx_conserto
        else:
            continue

total_por_linha = np.sum(Q, axis=1)
for i in range(Q.shape[0]):
    Q[i,i] = -total_por_linha[i]
Q

array([[ -6.,   1.,   1., ...,   0.,   0.,   0.],
       [  1., -37.,   1., ...,   0.,   1.,   0.],
       [  1.,   1., -37., ...,   1.,   0.,   0.],
       ...,
       [  0.,   0.,   1., ..., -37.,   1.,   1.],
       [  0.,   1.,   0., ...,   1., -37.,   1.],
       [  0.,   0.,   0., ...,   1.,   1.,  -6.]])

Calculando a distribuição Estacionária

In [59]:
from scipy.linalg import expm

PI = expm(Q*10)
pi = PI[0].round(10)
pi

array([0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625])

Agrupando por quantidade de arestas falhas (e somando entre os grupos)

In [60]:
dic = {}

for i, estado in enumerate(possibilities):
    if not len(estado) in dic.keys():
        dic[len(estado)] = pi[i]
    else:
        dic[len(estado)] += pi[i]

np.array(list(dic.values())).round(10)

array([0.015625, 0.09375 , 0.234375, 0.3125  , 0.234375, 0.09375 ,
       0.015625])

Calculando Recompensas

In [61]:
def esta_desconectado(grafo):
    tmp = np.where(grafo==-1, 0, grafo)
    G = nx.Graph(tmp)
    return not nx.is_connected(G)

def tem_link_desligado(grafo):
    i, _ = np.where(grafo == 1)
    return (6 - len(i)) > 0

def conectado_e_link_falho(grafo):
    G = nx.Graph(grafo)
    return tem_link_desligado(grafo) and nx.is_connected(G)

def gera_matriz_de_arestas(arestas, matriz_tam):
    m = np.zeros((matriz_tam,matriz_tam))

    for aresta in arestas:
        j, i = aresta
        m[i,j] = 1
        
    return m

def matriz_complementar(arestas_com_falha, tam):
    estado = []
    for aresta in arestas:
        if aresta in arestas_com_falha:
            continue
        estado.append(aresta)
    return gera_matriz_de_arestas(estado, tam)

In [11]:
_desconectado = 0
_sem_link = 0
_conectado_sem_link = 0

for i, estado in enumerate(possibilities):
    M = gera_matriz_de_arestas(estado, 5)

    _desconectado += esta_desconectado(M) * pi[i]
    _sem_link += tem_link_desligado(M) * pi[i]
    _conectado_sem_link += conectado_e_link_falho(M) * pi[i]

print("Desconectado:", _desconectado)
print("Conectado Sem link:", _conectado_sem_link)
print("Sem Link:", _sem_link)

Desconectado: 0.71875
Conectado Sem link: 0.265625
Sem Link: 0.984375


## Cenário 2 - Reparo Sequencial

In [81]:
arestas

EdgeView([(0, 1), (0, 4), (1, 2), (2, 3), (2, 4), (3, 4)])

In [88]:
import itertools
import networkx as nx

possibilities_seq = list()

for i in range(len(arestas)+1):
    for conjunto in itertools.permutations(arestas, i):
        possibilities_seq.append( list(conjunto) )

print(len(possibilities_seq))

1957


In [102]:
Q_seq = np.zeros((len(possibilities_seq),len(possibilities_seq)))

total_arestas = len(arestas)

for i in range(0, len(possibilities_seq)):
    estado_i = possibilities_seq[i]
    for j in range(0, len(possibilities_seq)):
        estado_j = possibilities_seq[j]

        if i == j:
            continue

        if estado_i[1:] == estado_j:
            Q_seq[i,j] = tx_conserto
        elif estado_j[:-1] == estado_i:
            Q_seq[i,j] = tx_falha


total_por_linha = np.sum(Q_seq, axis=1)
for i in range(Q_seq.shape[0]):
    if total_por_linha[i] > 0:
        Q_seq[i,i] = -total_por_linha[i]

Q_seq

Falha: [] [(0, 1)]
Falha: [] [(0, 4)]
Falha: [] [(1, 2)]
Falha: [] [(2, 3)]
Falha: [] [(2, 4)]
Falha: [] [(3, 4)]
Falha: [(0, 1)] [(0, 1), (0, 4)]
Falha: [(0, 1)] [(0, 1), (1, 2)]
Falha: [(0, 1)] [(0, 1), (2, 3)]
Falha: [(0, 1)] [(0, 1), (2, 4)]
Falha: [(0, 1)] [(0, 1), (3, 4)]
Falha: [(0, 4)] [(0, 4), (0, 1)]
Falha: [(0, 4)] [(0, 4), (1, 2)]
Falha: [(0, 4)] [(0, 4), (2, 3)]
Falha: [(0, 4)] [(0, 4), (2, 4)]
Falha: [(0, 4)] [(0, 4), (3, 4)]
Falha: [(1, 2)] [(1, 2), (0, 1)]
Falha: [(1, 2)] [(1, 2), (0, 4)]
Falha: [(1, 2)] [(1, 2), (2, 3)]
Falha: [(1, 2)] [(1, 2), (2, 4)]
Falha: [(1, 2)] [(1, 2), (3, 4)]
Falha: [(2, 3)] [(2, 3), (0, 1)]
Falha: [(2, 3)] [(2, 3), (0, 4)]
Falha: [(2, 3)] [(2, 3), (1, 2)]
Falha: [(2, 3)] [(2, 3), (2, 4)]
Falha: [(2, 3)] [(2, 3), (3, 4)]
Falha: [(2, 4)] [(2, 4), (0, 1)]
Falha: [(2, 4)] [(2, 4), (0, 4)]
Falha: [(2, 4)] [(2, 4), (1, 2)]
Falha: [(2, 4)] [(2, 4), (2, 3)]
Falha: [(2, 4)] [(2, 4), (3, 4)]
Falha: [(3, 4)] [(3, 4), (0, 1)]
Falha: [(3, 4)] [(3, 4), (0,

array([[-6.,  1.,  1., ...,  0.,  0.,  0.],
       [ 1., -6.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0., -6., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

In [100]:
from scipy.linalg import expm

PI_seq = expm(Q_seq*10)

pi_seq = PI_seq[0]
print(pi_seq, np.sum(pi_seq))

[0.00051149 0.00051141 0.00051141 ... 0.00051088 0.00051088 0.00051088] 0.9999999999999987


In [99]:
dic = {}

for i, estado in enumerate(possibilities_seq):
    if not len(estado) in dic.keys():
        dic[len(estado)] = pi_seq[i]
    else:
        dic[len(estado)] += pi_seq[i]

for i, value in enumerate(np.array(list(dic.values())).round(10)):
    print("Tempo com {} arestas com defeito: {}".format(i, value))

Tempo com 0 arestas com defeito: 0.0005114931
Tempo com 1 arestas com defeito: 0.0030684502
Tempo com 2 arestas com defeito: 0.0153392032
Tempo com 3 arestas com defeito: 0.0613441237
Tempo com 4 arestas com defeito: 0.1839938397
Tempo com 5 arestas com defeito: 0.3679102496
Tempo com 6 arestas com defeito: 0.3678326406


In [94]:
_desconectado = 0
_sem_link = 0
_conectado_sem_link = 0

for i, estado in enumerate(possibilities_seq):
    M = matriz_complementar(estado, 5)

    _desconectado += esta_desconectado(M) * pi_seq[i]
    _sem_link += tem_link_desligado(M) * pi_seq[i]
    _conectado_sem_link += conectado_e_link_falho(M) * pi_seq[i]

print("Desconectado:", _desconectado)
print("Conectado Sem link:", _conectado_sem_link)
print("Sem Link:", _sem_link)

Desconectado: 0.9851713078069119
Conectado Sem link: 0.014317199137823259
Sem Link: 0.9994885069447355
